# Determining FIB Wedge Angle Using t/Lamda Measurements

This is a personal workbook used to process my own data but has been made public for the use of others. If you find any errors or need further help using this workbook please contact k8macarthur@gmail.com. 

## Authors

Katherine E. MacArthur - Originally written for EMC2016, 28.08.2016

## Requirements

This code was written to work Hyperspy version 1.1 or later.

## 1. Importing Hyperspy and Libraries

Begin by importing the Hyperspy Function Library

In [1]:
import matplotlib
matplotlib.rcParams["backend"] = 'Qt4Agg'
import hyperspy.api as hs
import numpy as np
%matplotlib qt4

## 2. Importing and Aligning Data

Opens a load window. Use hs.load('filename') if filename is known specifically.

In [30]:
wedge = hs.load()
wedge.set_signal_type('EELS')
wedge.metadata.General.title = 'Original Data Wedge'
wedge

<EELSSpectrum, title: Original Data Wedge, dimensions: (200, 5|2048)>

Check current metadata.

In [31]:
wedge.metadata

├── Acquisition_instrument
│   └── TEM
│       ├── Detector
│       │   └── EELS
│       │       └── collection_angle = 19.379844665527344
│       ├── beam_energy = 200.0
│       ├── convergence_angle = 25.0
│       └── dwell_time = 0.002
├── General
│   ├── original_filename = EELS Spectrum Image.dm3
│   └── title = Original Data Wedge
└── Signal
    ├── binned = True
    └── signal_type = EELS

In [4]:
wedge.plot()
wedge.axes_manager.gui()

Adjust the metadata for all the sample parameters.

In [32]:
wedge.metadata.Acquisition_instrument.TEM.dwell_time = 1
wedge.axes_manager[0].scale = 64
wedge.axes_manager[1].scale = 64

Extract t over lambda using estimate_thickness(point), where point is the first minimum point after the zero loss peak.

In [33]:
wedge.align_zero_loss_peak()

100%|██████████| 1000/1000 [00:00<00:00, 6306.71it/s]


Initial ZLP position statistics
-------------------------------
Summary statistics
------------------
mean:	-0.897
std:	2.489

min:	-5.200
Q1:	-3.000
median:	-0.900
Q3:	1.300
max:	3.400



100%|██████████| 1000/1000 [00:00<00:00, 6246.65it/s]


In [47]:
wedge.isig[:150].spikes_removal_tool()

## 4. Extracting and Estimating Thickness Information

In [27]:
s = wedge.estimate_elastic_scattering_threshold()

In [35]:

wedge.plot()

In [51]:
thickness = wedge.estimate_thickness(5.5)
thickness.plot()

## 5. Converting from the Intensity Map to a Line Profile

This assumes that the line profile is parallel to the x-axis in the image.

In [68]:
intensity = thickness
width = intensity.axes_manager[0].scale
line_scan = np.zeros(intensity.axes_manager.signal_shape[0])
line_scan.astype(float)
for row in range(intensity.axes_manager.signal_shape[1]):
    for col in range(intensity.axes_manager.signal_shape[0]):
        line_scan[col] = line_scan[col] + intensity.data[row, col]
    
line_scan = line_scan/intensity.axes_manager.signal_shape[1]
x_data = np.zeros(intensity.axes_manager.signal_shape[0])
x_data.astype(float)
for i in range(intensity.axes_manager.signal_shape[0]):
    x_data[i] = i*width

The following function prints out the t/lambda thickness profile for import into another software e.g. excel and further processing.

In [11]:
wedge.save('Ni04EELS-processed')